### Imports and hyperparameters

In [1]:
import pandas as pd
import numpy as np

import os

from transformers import AutoTokenizer, TFAutoModel

import tensorflow as tf

In [16]:
# Hyperparameters
MAX_SEQUENCE_LENGTH = 100
EPOCHS = 5
BATCH_SIZE = 32
LEARNING_RATE = 3e-5
DROPOUT = 0.2

# Set a seed to decrease randomness
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

### Load demo dataset

In [18]:
# Enter the name of your demo file here
FILE_NAME = 'test.csv'

In [19]:
demo = pd.read_csv('test.csv')

### Set up tokenizer for DeBERTa model

In [20]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


### Tokenising and preparing inputs

In [30]:
# Encode data with a max length of 100
def bert_encode(hypotheses, premises, tokenizer, max_length=100):

    x = tokenizer(hypotheses, premises, padding='max_length', truncation=True, max_length=max_length)

    inputs = {
          'input_word_ids':tf.ragged.constant(x['input_ids']).to_tensor(),
          'input_mask': tf.ragged.constant(x['attention_mask']).to_tensor(),
          'input_type_ids': tf.ragged.constant(x['token_type_ids']).to_tensor()}

    return inputs

In [31]:
demo_input = bert_encode(demo.premise.values.tolist(), demo.hypothesis.values.tolist(), tokenizer)

### Define model

In [23]:
os.environ["WANDB_API_KEY"] = "0"

def build_model():
    bert_encoder = TFAutoModel.from_pretrained("microsoft/deberta-v3-base")
    input_word_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name="input_type_ids")

    output = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    output = tf.keras.layers.GlobalAveragePooling1D()(output)

    output = tf.keras.layers.Dropout(DROPOUT)(output)

    output = tf.keras.layers.Dense(1, activation='sigmoid')(output)

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)

    return model

In [24]:
model = build_model()
model.summary()

All model checkpoint layers were used when initializing TFDebertaV2Model.

All the layers of TFDebertaV2Model were initialized from the model checkpoint at microsoft/deberta-v3-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDebertaV2Model for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 100)]                0         []                            
 )                                                                                                
                                                                                                  
 input_mask (InputLayer)     [(None, 100)]                0         []                            
                                                                                                  
 input_type_ids (InputLayer  [(None, 100)]                0         []                            
 )                                                                                                
                                                                                            

In [25]:
# Load pretrained model
model.load_weights('best.model.weights.h5')

/usr/local/lib/python3.11/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


### Compute predictions

In [33]:
outputs = model.predict(demo_input)

preds = outputs > 0.5

104/104 [==============================] - 37s 260ms/step


In [34]:
demo['prediction'] = preds.astype(int).reshape(-1)

In [35]:
# Display the first 20 predictions
demo.head(20)

,premise,hypothesis,prediction
0,"Boy wearing red hat, blue jacket pushing plow ...",The boy is surrounded by snow,1
1,A blond woman in a black shirt is standing beh...,The woman is standing.,1
2,Three people in uniform are outdoors and are o...,Uniformed people are outside,1
3,"A person, in a striped blue shirt and pants, i...",The person is running,1
4,"A man, woman, and child get their picture take...",A family on vacation is posing.,1
5,A tennis player in blue shorts and a white shi...,The person is taking a nap in their bed.,0
6,A boy looks on at an electric device with thre...,A boy is taking a nap in his bed.,0
7,A young girl is standing in a kitchen holding ...,A boy is playing with a firetruck.,0
8,Three men in yellow green safety vests sit on ...,People are playing volleyball on the sand.,0
9,man dressed in orange clothing with face cover...,Two men are in a shopping mall.,1


### Saving predictions only

In [36]:
preds_df = pd.DataFrame({'prediction': preds.astype(int).reshape(-1)})

preds_df.to_csv('predictions.csv', index=False)